## Laboratorium 7.2


## Przepływ optyczny i śledzenie obiektów

### Wprowadzenie

Metodę przepływu optycznego (między innymi) można stosować również do śledzenia obiektów. Zazwyczaj wykorzystuje się w tym celu metody rzadkie połączone ze znanymi już nam metodami detekcji punktów zainteresowania. Procedura przebiega podobnie jak w przypadku gęstego przepływu (tzn. operujemy na parze klatek w skali szarości, znajdujemy wektory przesunięć), z tą różnicą że obliczanie przepływu zachodzi wyłącznie dla interesujących nas punktów - a zatem musimy znać ich położenia, aby zainicjować proces. Można sobie wyobrazić różne sposoby zlokalizowania punktów kluczowych, w zależności od konkretnego zastosowania i założeń: być może np. pipeline do przetwarzania rozpoczyna się od zlokalizowania jakichś obiektów, być może z góry wiemy, gdzie obiekty się znajdują.

W OpenCV podstawową metodą do obliczania przepływu dla zbioru punktów jest metoda Lucasa-Kanade, zaimplementowana w funkcji [`cv2.calcOpticalFlowPyrLK`](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323). Podobnie jak w metodzie Farnebacka, argument `nextPts` ustawiamy na `None` (chyba, że chcemy zainicjować rozwiązanie już znanymi położeniami punktów, ale w praktyce z reguły nie będziemy mieli takiej wygody), argumenty `status` i `err` pomijamy zupełnie. W odróżnieniu od Farnebacka, wynikiem są absolutne położenia punktów na drugiej klatce (a nie wektory przesunięć). Dodatkowo otrzymujemy też wektor `status` informujący dla każdego punktu, czy udało się go odnaleźć, oraz `err` mierzący jakąś miarę błędu wizualnego pomiędzy fragmentem obrazu zawierającym dany punkt kluczowy.

Do inicjalizacji rozwiązania można wykorzystać znane nam metody znajdowania punktów kluczowych, albo opartą o detektor Harrisa funkcję [`cv2.goodFeaturesToTrack`](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541) - jest nieco prostsza w użyciu, bo na wyjściu otrzymujemy po prostu macierz `np.float32` o wymiarach $Nx1x2$ zawierającą koordynaty znalezionych punktów.

Oczywiście, śledzenie obiektów oparte o jednorazową inicjalizację jest narażone na błędy: zniknięcie lub tymczasowe zasłonięcie śledzonego obiektu, czy choćby zmiana jego wyglądu (np. wskutek pojawienia się cienia) mogą spowodować, że przepływ nie zostanie obliczony. Może być też pożądana możliwość śledzenia ciągłego z uwzględnieniem pojawiania się nowych obiektów (np. monitorowanie CCTV). W takich sytuacjach konieczne jest regularne odświeżanie inicjalizacji, tj. ponowne wykrycie interesujących obiektów. To z kolei prowadzi do kolejnego problemu: jeżeli interesuje nas cała historia ruchu danych obiektów, przy czym do pewnego momentu ich lokalizacje uzyskiwaliśmy za pomocą przepływu optycznego, a w pewnym momencie wykonaliśmy ponowną detekcję - to jak powiązać jedno z drugim, by zapewnić ciągłość lokalizacji? To zagadnienie nie będzie jednak centralne na laboratorium.

---

In [ ]:
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

### Zadanie 1

#### Zadanie 1a

Dany jest plik `amsterdam47.mp4` przedstawiający zapis ruchu na skrzyżowaniu. Uzupełnij podstawową pętlę analizy wideo o śledzenie obiektów: inicjalizację i wykorzystanie przepływu optycznego do aktualizacji ich położenia. Celem jest pozyskanie pełnej trajektorii obiektów, tak aby można było ją narysować (np. [`cv2.polylines`](https://docs.opencv.org/3.4/d6/d6e/group__imgproc__draw.html#gaa3c25f9fb764b6bef791bf034f6e26f5)) tudzież przetwarzać dalej.

Wynik zaprezentuj w postaci graficznej (narysowane trajektorie znalezionych obiektów). Niekoniecznie musisz przetwarzać cały materiał wideo - zwróć uwagę, że pojazdy znikają poza klatką, uniemożliwiając ich dalsze śledzenie.

In [ ]:
def baseLoop(source:cv2.VideoCapture):
    frame1 = None
    status, frame2 = source.read()
    frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # inicjalizacja
    while status:
        frame1 = frame2
        status, frame2 = source.read()
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        # przetwarzanie

#### Zadanie 1b

Powtórz poprzednie doświadczenie na różnych fragmentach materiału źródłowego. Kiedy i w jaki sposób metoda zawodzi?

Komentarz 1:

...

### Zadanie 2

Wykorzystując historię położenia poszczególnych obiektów, zidentyfikuj punkty należące do tła i odfiltruj je ze zbioru śledzonych obiektów.  
*Jeśli starczy czasu:* zidentyfikuj punkty należące do tych samych obiektów (tj. poruszające się w ten sam sposób).

---

### Zadanie ekstra

Rozszerz rozwiązanie z zadania 1 o możliwość śledzenia nowych obiektów pojawiających się w polu widzenia. Zadbaj o to, by ponowna detekcja punktów zainteresowania nie tworzyła duplikatów obiektów, które już są znane i śledzone.

In [ ]:
# zadanie dodatkowe, nie ma za nie extra punktow